In [32]:
pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [33]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [34]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
        verify_ssl=False,
        extra_headers={
            'X-DESTINATION': 'Meld',
            'X-POU': 'TREAT',
            # 'Host': 'concept01.ehealthexchange.org:52780'
        })

    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'

    return fhir_client

In [35]:
client = setup_fhir_client()

search_results = client.search('Patient', {
    'family': 'Blackstone',
    'given': 'Veronica'
})
Pretty(json.dumps(search_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "5d9578f3-f6ef-44c0-8e89-c5fbfb847eb7",
    "meta": {
        "lastUpdated": "2024-01-16T19:27:18.445+00:00"
    },
    "type": "searchset",
    "total": 1,
    "link": [
        {
            "relation": "self",
            "url": "https://gw.interop.community/HeliosConnectathonSa/data/Patient?family=Blackstone&given=Veronica"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/101h",
            "resource": {
                "resourceType": "Patient",
                "id": "101h",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2024-01-16T15:10:12.000+00:00",
                    "source": "#egQALPFh5OgeIPvd",
                    "profile": [
                        "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
                    ]
                },
                "text": {
                  

There should be exactly one `Patient` matching the search.

In [36]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

{
    "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/101h",
    "resource": {
        "resourceType": "Patient",
        "id": "101h",
        "meta": {
            "versionId": "1",
            "lastUpdated": "2024-01-16T15:10:12.000+00:00",
            "source": "#egQALPFh5OgeIPvd",
            "profile": [
                "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
            ]
        },
        "text": {
            "status": "generated",
            "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">This is a simple narrative with only plain text</div>"
        },
        "extension": [
            {
                "url": "http://mihin.org/extension/copyright",
                "valueString": "Copyright 2014-2023 Michigan Health Information Network Shared Services. Licensed under the Apache License, Version 2.0 (the 'License'); you may not use this file except in compliance with the License. You may obtain a copy of the Licens

In [37]:
HTML(patient['resource']['text']['div'])

In [38]:
everything = client.everything('Patient', patient['resource']['id'], count=1000)
Pretty(json.dumps(everything, indent=4))

{
    "resourceType": "Bundle",
    "id": "ec789088-0656-4357-987b-cc45790312cf",
    "meta": {
        "lastUpdated": "2024-01-16T19:27:18.827+00:00"
    },
    "type": "searchset",
    "total": 15,
    "link": [
        {
            "relation": "self",
            "url": "https://gw.interop.community/HeliosConnectathonSa/data/Patient/101h/$everything?_count=1000"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/101h",
            "resource": {
                "resourceType": "Patient",
                "id": "101h",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2024-01-16T15:10:12.000+00:00",
                    "source": "#egQALPFh5OgeIPvd",
                    "profile": [
                        "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
                    ]
                },
                "text": {
                    "

`MedicationAdministration` should have the medications administered to this Patient.

In [39]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id'],
    'code': '2671695' # penicillin
})
Pretty(json.dumps(medication_administration_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "d0580528-06ef-4d3c-9112-08885f91c2cf",
    "meta": {
        "lastUpdated": "2024-01-16T19:27:19.364+00:00"
    },
    "type": "searchset",
    "total": 1,
    "link": [
        {
            "relation": "self",
            "url": "https://gw.interop.community/HeliosConnectathonSa/data/MedicationAdministration?code=2671695&patient=101h"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/MedicationAdministration/123M",
            "resource": {
                "resourceType": "MedicationAdministration",
                "id": "123M",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2024-01-16T15:15:04.000+00:00",
                    "source": "#Zb5pCzy4r8r4yS6a"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">This 

`Observation` should have lab test results, and pregnancy status.

In [40]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id'],
    'code': '82810-3' # Pregnancy status
})

Pretty(json.dumps(observation_search_results, indent=4))        

{
    "resourceType": "Bundle",
    "id": "22b0521f-7aff-4457-be16-93dfcb5ce032",
    "meta": {
        "lastUpdated": "2024-01-16T19:27:19.738+00:00"
    },
    "type": "searchset",
    "total": 1,
    "link": [
        {
            "relation": "self",
            "url": "https://gw.interop.community/HeliosConnectathonSa/data/Observation?code=82810-3&patient=101h"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Observation/115H",
            "resource": {
                "resourceType": "Observation",
                "id": "115H",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2024-01-16T15:15:29.000+00:00",
                    "source": "#VOnRCYAYZPcAXOId"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">This is a simple narrative with only plain t

`Encounter` should have hospitalization status.

In [41]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "64350079-9012-4f5c-880b-1a85d828675e",
    "meta": {
        "lastUpdated": "2024-01-16T19:27:20.116+00:00"
    },
    "type": "searchset",
    "total": 1,
    "link": [
        {
            "relation": "self",
            "url": "https://gw.interop.community/HeliosConnectathonSa/data/Encounter?patient=101h"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Encounter/106H",
            "resource": {
                "resourceType": "Encounter",
                "id": "106H",
                "meta": {
                    "versionId": "1",
                    "lastUpdated": "2024-01-16T15:13:00.000+00:00",
                    "source": "#FMsZeOwElJGYmXbM"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">This is a simple narrative with only plain text</div>"
        

In [42]:
client.close()